In [116]:
import json
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

### get conference rankings
- from https://www.resurchify.com/conference-ranking

In [108]:
def scrape_conference_rankings():
    # get soup
    r = requests.get("https://www.resurchify.com/conference-ranking")
    assert r.status_code == 200
    soup  = BeautifulSoup(r.text, "html.parser")
    # parse
    all_conference_info = []
    all_conferences = soup.find_all("tr", {"class":"w3-hover-sand w3-bordered"})
    for conference_info in all_conferences:
        all_conference_info.append([span.text for span in conference_info.find_all("span")])
    return pd.DataFrame(all_conference_info, columns = ["publication", "acronym", "rank"])

In [109]:
all_conference_info = scrape_conference_rankings()

In [118]:
all_conference_info.head(10)

,publication,acronym,rank
0,ACIS Conference on Software Engineering Resear...,SERA,C
1,"ACM Conference on Applications, Technologies, ...",SIGCOMM,A*
2,ACM Conference on Computer and Communications ...,CCS,A*
3,ACM Conference on Computer Supported Cooperati...,CSCW,A
4,ACM Conference on Economics and Computation,EC,A*
5,ACM Conference on Embedded Networked Sensor Sy...,SENSYS,A*
6,ACM Conference on Embedded Software,EMSOFT,A
7,ACM Conference on Object Oriented Programming ...,OOPSLA,A*
8,ACM Information Technology Education,SIGITE,National: USA
9,ACM International Conference on Advances in Co...,ACE,B


### save

In [114]:
import pickle
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 2\data\conference_ranks.pkl"
all_conference_info.to_pickle(path)

In [ ]:
data_dir = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 2\data\google scholar profiles"
os.chdir(data_dir)
for filename in os.listdir():
    with open(filename, "r") as f:
        data = json.load(f)
def get_publication_acronym(string):
    return re.findall(r'\(.*?\)', string)[0][1:-1]
get_publication_acronym('AAAI Conference on Artificial Intelligence (AAAI), 2016')

### initially, i could also acquire the h index and impact scores with the following code, however, the search function in www.resurchify.com became spoiled somehow, the conferences can longer be searched

In [31]:
import requests
from bs4 import BeautifulSoup

def get_soup(conference_name):
    query = "+".join(conference_name.split())
    url = f"https://www.resurchify.com/find/?query={query}"
    r = requests.get(url)
    assert r.status_code == 200
    soup  = BeautifulSoup(r.text, "html.parser")
    return soup

In [32]:
def get_stats(soup, search_name):
    try:
        # from the conference search results find the right one 
        conference_search_results = soup.find_all("div", {"class":"w3-white w3-container w3-card-4 w3-hover-light-gray"})
        if len(conference_search_results)==0:
            return
        elif len(conference_search_results)==1:
            conference_info = conference_search_results[0]
        elif len(conference_search_results)>1:
            for conference_info in conference_search_results[:5]:
                conference_name = conference_info.find("h3").text
                if score(conference_name, search_name) == 100:
                    break
            else:
                return 


        conference_impact = {}
        for stats in conference_info.find_all("span"):
            tmp_stats = stats.text.strip().split(":")
            conference_impact[tmp_stats[0]] = float(tmp_stats[1])
        return conference_impact
    except:
        return 
    

In [ ]:
from fuzzywuzzy import fuzz
"""
Flexible string matching. 
For instance, the following two strings can match
"1. Proceedings of the IEEE Computer Society Conference on Computer Vision and Pattern Recognition" 
"IEEE Conference on Computer Vision and Pattern Recognition"
"""
def score(s1, s2):
    scores = [
        fuzz.token_sort_ratio(s1, s2), 
        fuzz.token_set_ratio(s1, s2), 
        fuzz.partial_ratio(s1, s2)
    ]
    return max(scores)

In [ ]:
soup = get_soup("Computer Vision and Pattern Recognition",  "IEEE Conference on Computer Vision and Pattern Recognition")
stats = get_stats(soup,  "IEEE Conference on Computer Vision and Pattern Recognition")